In [1]:
from main import *
s = login()


b'{"user":{"id":"YW10763"},"token":{"expiry":21600.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'


In [2]:

fo_tracker = get_alphas("11-11 12:50:00", "11-20 17:40:00", 0.7, 0.4, "EUR", 100, "tarck") 
print("剪枝前的数量：\n")
print(len(fo_tracker))
print("开始剪枝\n")


b'{"user":{"id":"YW10763"},"token":{"expiry":21600.0},"permissions":["BEFORE_AND_AFTER_PERFORMANCE_V2","BRAIN_LABS","BRAIN_LABS_JUPYTER_LAB","CONSULTANT","MULTI_SIMULATION","PROD_ALPHAS","REFERRAL","SUPER_ALPHA","VISUALIZATION","WORKDAY"]}'
0
{'count': 6003, 'next': 'http://api.worldquantbrain.com:443/users/self/alphas?dateCreated%3C=2025-11-20T04%3A40%3A00-05%3A00&dateCreated%3E=2025-11-10T23%3A50%3A00-05%3A00&hidden=false&is.fitness%3E0.4=&is.sharpe%3E0.7=&limit=100&offset=100&order=-is.sharpe&settings.region=EUR&status=UNSUBMITTED%1FIS_FAIL&type%21=SUPER', 'previous': None, 'results': [{'id': 'e7wxLg3z', 'type': 'REGULAR', 'author': 'YW10763', 'settings': {'instrumentType': 'EQUITY', 'region': 'EUR', 'universe': 'TOP1200', 'delay': 1, 'decay': 24, 'neutralization': 'STATISTICAL', 'truncation': 0.08, 'pasteurization': 'ON', 'unitHandling': 'VERIFY', 'nanHandling': 'ON', 'maxTrade': 'OFF', 'language': 'FASTEXPR', 'visualization': False, 'startDate': '2013-01-20', 'endDate': '2023-01-2

In [3]:

def prune(next_alpha_recs, prefix, keep_num):
    from collections import defaultdict

    # 新结构: [alpha_id, exp, sharpe, turnover, fitness, margin, returns, drawdown, dateCreated, decay]
    # 索引说明:
    # 0: alpha_id
    # 1: exp
    # 2: sharpe
    # 3: turnover
    # 4: fitness
    # 5: margin
    # 6: returns
    # 7: drawdown
    # 8: dateCreated
    # 9: decay

    # Step 1: 过滤包含 prefix 的记录
    filtered_recs = [rec for rec in next_alpha_recs if prefix in rec[1]]

    # Step 2: 按 field 分组
    groups = defaultdict(list)
    for rec in filtered_recs:
        exp = rec[1]
        # 提取 field：去掉 prefix 后，取第一个逗号前的部分
        field_part = exp.split(prefix, 1)[-1]  # split only once
        field = field_part.split(",")[0]

        sharpe = rec[2]
        # 如果 sharpe < 0，则 field 加负号（保持原逻辑）
        if sharpe < 0:
            field = "-" + field

        # 解包关键指标
        returns = rec[6]
        drawdown = rec[7]

        # 新增条件：returns 必须大于 drawdown
        returns = float(returns)
        drawdown = float(drawdown)
        if returns <= drawdown:
            continue  # 不满足条件，跳过

        # 将完整记录加入对应 field 组
        groups[field].append(rec)

    # Step 3: 对每组按多目标排序，取 top keep_num
    output = []
    for field, recs in groups.items():
        # 多关键字排序：
        # 主要：sharpe ↓, fitness ↓, margin ↓, returns ↓, drawdown ↑（即越小越好）
        # Python 排序默认升序，所以对 drawdown 取负或用 reverse 配合元组
        # 更清晰的方式：使用 key 返回一个 tuple，其中 drawdown 取负以实现“越小越好”
        sorted_recs = sorted(
            recs,
            key=lambda r: (
                r[2],      # sharpe (maximize → descending)
                r[4],      # fitness (maximize)
                r[5],      # margin (maximize)
                r[6],      # returns (maximize)
                -r[7]      # drawdown (minimize → 所以用负号，这样大的 -drawdown 表示小的 drawdown)
            ),
            reverse=True  # 所有目标都希望越大越好（drawdown 已处理为 -drawdown）
        )

        # 取前 keep_num 个
        top_recs = sorted_recs[:keep_num]
        for rec in top_recs:
            output.append([rec[1], rec[-1]])  # [exp, decay]

    return output

fo_layer = prune(fo_tracker, 'mdl25_', 5)

In [5]:

print("新剪枝后的数量：")
print(len(fo_layer))
print(fo_layer)


新剪枝后的数量：
4
[['group_zscore(ts_product(winsorize(ts_backfill(mdl25_history_9v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 8], ['group_max(ts_product(winsorize(ts_backfill(mdl25_history_9v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 6], ["group_mean(-convert(winsorize(ts_backfill(mdl25_rdv421_6v, 120), std=4), mode = 'dollar2share'),1,densify(sta1_allc10))", 10], ['group_zscore(ts_product(winsorize(ts_backfill(mdl25_history_21v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 6]]


In [7]:
# 旧剪枝
def prune(next_alpha_recs, prefix, keep_num):
    from collections import defaultdict
    output = []
    num_dict = defaultdict(int)

    # 先过滤：只保留含 prefix 的记录
    filtered_recs = [rec for rec in next_alpha_recs if prefix in rec[1]]

    for rec in filtered_recs:
        exp = rec[1]
        field = exp.split(prefix)[-1].split(",")[0]
        sharpe = rec[2]
        if sharpe < 0:
            field = "-" + field
        if num_dict[field] < keep_num:
            num_dict[field] += 1
            decay = rec[-1]
            output.append([exp, decay])
    return output
fo_layer = prune(fo_tracker, 'mdl25_', 5)
print("旧剪枝后的数量：")
print(len(fo_layer))
print(fo_layer)


旧剪枝后的数量：
8
[['group_zscore(ts_product(winsorize(ts_backfill(mdl25_history_9v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 8], ["group_mean(-convert(winsorize(ts_backfill(mdl25_rdv421_6v, 120), std=4), mode = 'dollar2share'),1,densify(sta1_allc10))", 10], ['group_zscore(ts_product(winsorize(ts_backfill(mdl25_history_21v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 6], ['group_max(ts_product(winsorize(ts_backfill(mdl25_history_9v, 120), std=4), 20),densify(oth455_partner_roam_w5_pca_fact3_cluster_20))', 6], ["-group_neutralize(-convert(winsorize(ts_backfill(mdl25_rdv421_6v, 120), std=4), mode = 'dollar2share'),densify(sta1_allc10))", 10], ["-group_mean(-convert(winsorize(ts_backfill(mdl25_rdv421_6v, 120), std=4), mode = 'dollar2share'),1,densify(oth455_relation_n2v_p10_q200_w5_kmeans_cluster_20))", 10], ["-group_neutralize(-convert(winsorize(ts_backfill(mdl25_rdv421_6v, 120), std=4), mode = 'dollar2share'),densify(oth455_relatio